In [ ]:
import pandas as pd
from google.colab import files

# Upload CSV file manually in Colab
uploaded = files.upload()

Saving BakedFoodNutritions.csv to BakedFoodNutritions (1).csv


In [ ]:
# Load the dataset
df = pd.read_csv("BakedFoodNutritions.csv")

# Fill missing values with median
df.fillna(df.median(numeric_only=True), inplace=True)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Keep only important features
selected_features = ["Calories-kcl", "Protein-g", "Carb-g", "Sugar-g", "SaturatedFat-g"]
df = df[selected_features].copy()  # Ensure we are working on a copy

# Normalize Data
scaler = MinMaxScaler()
df.loc[:, selected_features] = scaler.fit_transform(df[selected_features])


In [ ]:
# New HealthScore formula based on only 5 features
df["HealthScore"] = (
    df["Protein-g"] * 2 -     # More protein = healthier
    df["Sugar-g"] * 1.5 -     # More sugar = unhealthier
    df["SaturatedFat-g"] * 0.1  # More fat = unhealthier
)

# Classify Healthy or Unhealthy (Threshold = 0.3)
df["Healthy"] = (df["HealthScore"] > df["HealthScore"].quantile(0.45)).astype(int)
# df["Healthy"] = (df["HealthScore"] > df["HealthScore"].median()).astype(int)



# Check distribution
df["Healthy"].value_counts()


,count
Healthy,
1,361
0,295


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define X and y
X = df[selected_features]
y = df["Healthy"]

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Simple Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate Model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy:.2%}")


✅ Model Accuracy: 96.97%


In [ ]:
import joblib

joblib.dump(rf_model, "baked_food_rf_final_v3.pkl")
files.download("baked_food_rf_final_v3.pkl")

